# Evaluation Script

Follow script implement foundamental steps for evaluate a Language translation system using Prometheus Eval like as Judge

In [1]:
import torch
import pandas as pd
from PrometheusAPI import PrometheusJudge
from PrometheusAPI import MPrometheusJudge
from datasets import Dataset
from io import StringIO
from utils import jsonline, txtline
from sklearn.metrics import cohen_kappa_score

In [36]:
DEVICE = ("cuda" if torch.cuda.is_available() else "cpu")
groupname = "Many_Napts_Little_Progress"
criterio = "standard"
model_name = "Opus"

### Prepare Prometheus-as-a-Judge 🔥

In [3]:
j = PrometheusJudge(DEVICE)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

### Load Data to evaluate

In [4]:
title = "opus-mt-itc-itc(opus).csv"
dataset = Dataset.from_csv(title)


In [5]:
dataset["Prompt"]

['>>ita<< Altressì uno amante chiamando merzé alla sua donna dice parole e ragioni molte, et ella si difende in suo dire.',
 ">>ita<< A Milano fue ripressa la malvagità d' una donna in simile bugìa, nel tempo medesimo di questo signore della republica, in questo modo:",
 '>>ita<< Nella quale battaglia, certo io ebbi sempre ardire di ragionare di pace e sempre mi dolfi che non solamente la pace era schifata,',
 ">>ita<< sono due già non in una carne, ma in uno spirito, cioè Iddio, e l' anima. Onde in altro luogo dice S. Paolo: Chi s' accosta a Dio è uno spirito",
 ">>ita<< Il re entrò in uno giardino dietro al suo albergo, quasi come s'egli andasse pensando alla risposta.",
 '>>ita<< Gorgone, e ho questa proprietà che io volo per l\'aire sì come uno ucello".',
 '>>ita<< Gregorio. Non udii mai che egli avesse maestro; ma il dono dello Spirito Santo non si può stringere a legge.',
 ">>ita<< e l' acconciamento a fare grandissime cose, cioè a ttenere pace et amare Idio e 'l proximo, a ffare

### Run Judge and Record the Response

In [6]:
tokenized = j.tokenize(dataset)
j.get_tokenizer().batch_decode(tokenized["input_ids"], skip_special_tokens=True)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

['You are a fair judge assistant tasked with providing clear, objective feedback based on specific criteria, ensuring each assessment reflects the absolute standards set for performance. \n\n ###Task Description:\n        An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score from 1 to 5, and a score rubric representing an evaluation criteria are given.\n        1. Write a detailed feedback that assesses the quality of the response strictly based on the given score rubric, not evaluating in general.\n        2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.\n        3. The output format should look as follows: "Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)"\n        4. Please do not generate any other opening, closing, and explanations.\n\n        ###The instruction to evaluate:\n        >>ita<< Altressì uno amante chiamand

In [7]:
r = j.judge(tokenized)


🔥⚖️Judging⚖️🔥:   0%|          | 0/10 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [12]:
txtline(r, "PrometheusEvalJudge.txt")
scores = j.parse_responses(r)

### Annotate Results

In [35]:
ann = pd.read_csv(title, delimiter=",", quoting=1)
ann["Judge_Score(Prometheus)"] = scores

### CSV Report (Fast Revision)

In [37]:
ann.to_csv(f"{groupname}-hw2_transl-{model_name}")

### Jsonl Report (Requirements)

### Human-Metrics Correlation

In [15]:
judges = ["Judge_Score(Prometheus)", "Judge_Score(Gemini)", "Judge_Score(GPT)"]
human = ["User_Score"]

In [ ]:
from sklearn.metrics import mean_absolute_error


for j, h in zip(judges, human*3):
    y1 = ann[j]
    y2 = ann[h]
  
    cks = cohen_kappa_score(y1, y2, labels=[1,2,3,4,5], weights="quadratic")
    mar = mean_absolute_error(y2,y1)
  
    print(cks)
    print(mar)




-0.31034482758620685
1.6
3.8
0.5454545454545454
1.1
1.5
0.3846153846153846
1.4
2.4
